In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

In [ ]:
spark = SparkSession.builder \
    .master("local") \
    .appName("pyspark_giris") \
    .getOrCreate()
    
sc = spark.sparkContext

In [ ]:
sc

In [ ]:
spark_df = spark.read.csv("../input/churn-predictions-personal/Churn_Predictions.csv", header = True, inferSchema = True)

In [ ]:
spark_df

In [ ]:
type(spark_df)

In [ ]:
spark_df.printSchema()

In [ ]:
spark_df.head()

In [ ]:
spark_df.dtypes

In [ ]:
spark_df.show(3, truncate = True)

In [ ]:
spark_df.count()

In [ ]:
spark_df.columns

In [ ]:
spark_df.describe().show()

In [ ]:
spark_df.describe("Age").show()

In [ ]:
spark_df.select("Age","Surname").show()

In [ ]:
spark_df.filter(spark_df.Age > 40).count()

In [ ]:
spark_df.filter(spark_df.Age > 40).show()

In [ ]:
spark_df.groupby("Exited").count().show()

In [ ]:
spark_df.groupby("Exited").agg({"Age":"mean"}).show()

In [ ]:
spark_df.createOrReplaceTempView("tbl_df")

In [ ]:
spark.sql("show databases").show()

In [ ]:
spark.sql("show tables").show()

In [ ]:
spark.sql("select Age from tbl_df").show(5)

In [ ]:
spark.sql("select Exited, mean(Age) from tbl_df group by Exited").show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.barplot(x = "Exited", y = spark_df.Exited.index, data = spark_df)

In [ ]:
spark_df.groupby("Exited").agg({"Age":"mean"}).show()

In [ ]:
spark_df.groupby("Exited").agg({"Age":"mean"}).toPandas()

In [ ]:
sdf = spark_df.toPandas()

sns.barplot(x = "Exited", y = sdf.Exited.index, data = sdf);

In [ ]:
# GBM ile Müşteri Terk Modellemesi

In [ ]:
spark_df = spark.read.csv("../input/churn-predictions-personal/Churn_Predictions.csv", header = True,inferSchema = True)

In [ ]:
spark_df.show(5)

In [ ]:
spark_df = spark_df.toDF(*[c.lower() for c in spark_df.columns])

In [ ]:
spark_df.show(5)

In [ ]:
spark_df = spark_df.withColumnRenamed("_c0", "index")

In [ ]:
spark_df.show(5)

In [ ]:
spark_df.count()

In [ ]:
len(spark_df.columns)

In [ ]:
spark_df.describe().show()

In [ ]:
spark_df.select("age","creditscore","balance").describe().toPandas().transpose()

In [ ]:
spark_df = spark_df.dropna()

In [ ]:
spark_df = spark_df.withColumn("age_kare", spark_df.age**2)

In [ ]:
spark_df.show()

In [ ]:
#bagimli degiskeni belirtmek 

from pyspark.ml.feature import StringIndexer

In [ ]:
stringIndexer = StringIndexer(inputCol = "exited", outputCol = "label")

In [ ]:
mod = stringIndexer.fit(spark_df)

In [ ]:
indexed = mod.transform(spark_df)

In [ ]:
spark_df = indexed.withColumn("label", indexed["label"].cast("integer"))

In [ ]:
spark_df.show()

In [ ]:
#bagimsiz degiskenler

from pyspark.ml.feature import VectorAssembler

In [ ]:
spark_df.columns

In [ ]:
cols = ['creditscore','age','tenure','balance','numofproducts','hascrcard','isactivemember','estimatedsalary','age_kare']

In [ ]:
va = VectorAssembler(inputCols = cols, outputCol = "features")

In [ ]:
spark_df.show()

In [ ]:
va_df = va.transform(spark_df)

In [ ]:
va_df.show()

In [ ]:
final_df = va_df.select(["features","label"])

In [ ]:
final_df.show()

In [ ]:
splits = final_df.randomSplit([0.70, 0.30])
train_df = splits[0]
test_df = splits[1]

In [ ]:
test_df.show()

In [ ]:
#base model

from pyspark.ml.classification import GBTClassifier

In [ ]:
gbm = GBTClassifier(maxIter = 10, featuresCol = "features", labelCol = "label")

gbm_model = gbm.fit(train_df)

sc

y_pred = gbm_model.transform(test_df)

In [ ]:
sc

In [ ]:
y_pred.show(20)

In [ ]:
ac = y_pred.select("label","prediction")

ac.show(3)

In [ ]:
ac.filter(ac.label == ac.prediction).count()/ac.count()

In [ ]:

#model tuning

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

evaluator = BinaryClassificationEvaluator()

paramGrid = (ParamGridBuilder()
             .addGrid(gbm.maxDepth, [2, 4, 6])
             .addGrid(gbm.maxBins, [20, 30])
             .addGrid(gbm.maxIter, [10, 20])
             .build())

cv = CrossValidator(estimator = gbm, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = 10)

cv_model = cv.fit(train_df)

y_pred = cv_model.transform(test_df)

ac = y_pred.select("label","prediction")

ac.filter(ac.label == ac.prediction).count() / ac.count()

In [ ]:
# yeni ya da var olan bir musterimiz var. Bu musteri bizi terk eder mi?

import pandas as pd

names = pd.Series(["Ali Ahmetoğlu", "Taner Gün", "Berkay","Polat Konak", "Kamil Atasoy"])
crScrs = pd.Series([370,680,710,800,570])
ages = pd.Series([33,36,38,27,41])
tenures = pd.Series([3,5,5,2,4])
balances = pd.Series([0,3000,67532,325455,4400])
numofprods = pd.Series([1,2,2,3,4])
crCards = pd.Series([0,1,1,1,0])
actives = pd.Series([1,1,0,1,0])
estSalaries = pd.Series([27000,68000,92000,120000,43000])
ageKares = pd.Series([33**2,36**2,38**2,27**2,41**2])

cols = ['creditscore','gender','age','tenure','balance','numofproducts','hascrcard','isactivemember','estimatedsalary','age_kare']

In [ ]:
yeni_musteriler = pd.DataFrame({
    'names':names,
    'creditscore':crScrs,
    'age': ages,
    'tenure': tenures ,
    'balance': balances,
    'numofproducts':numofprods,
    'hascrcard':crCards,
    'isactivemember':actives,
    'estimatedsalary':estSalaries,
    'age_kare':ageKares})

In [ ]:
yeni_musteriler.columns

In [ ]:
yeni_musteriler.head()

In [ ]:

yeni_sdf = spark.createDataFrame(yeni_musteriler)


In [ ]:
yeni_sdf.show(3)

In [ ]:
new_customers = va.transform(yeni_sdf)

new_customers.show(3)

In [ ]:
results = cv_model.transform(new_customers)

results.select("names","prediction").show()

In [ ]:
sc.stop()